<html>
    <body>
        <h1 align='center'> Motion Detection </h1>
            <p> For this task, I used the same technique of setting the first frame as background and calculating the difference between that and the current frame to get the movement. However, this time I just needed cars going to the city center so I knew that every car going there had to pass through the road the top part of the main road. So I cropped the frame so that I can just get about the width of 1 car. Then whenever movement was detected in this part, I set the current car numbers to 1. I then set the previous cars with the value of the current cars on the previous frame. Lastly, I used if condition to check if the previous car was greater than current car (car left that area of screen) so then I increased the car count by 1. </p>
    </body>
</html>

In [2]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import datetime

In [4]:
def car_detection(video_name):

    #set first frame to None initially 
    first_frame = None

    #set previous cars to none
    previous_cars = None

    # We use VideoCapture function to open video
    video=cv2.VideoCapture(video_name + ".mp4")

    #initially set the cars to 0
    car_count=0

    # count the number of frames and fps to get the time
    frame_num = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = video.get(cv2.CAP_PROP_FPS)
  
    # calculate duration of the video
    seconds = round(frame_num / fps)
    seconds = seconds % 3600
    minutes = round(seconds / 60,2)

    # Start a loop and keep reading frames until we press 'q' or the video ends
    while True:
    
        #reads frames
        check, frame = video.read()
    
        #Checks if the video has ended and breaks the loop if its true
        if frame is None:
            break
 
        #First convert to grey because opencv uses brg and crop it to the needed location
        current_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)[320:380 , 60:105]
    
        #Checks if the first frame is not assigned yet
        if (first_frame is None):
            #Assigns that frame as the first frame
            first_frame = current_frame
            continue
            
        #Calculates the difference and threshold
        delta_frame = cv2.absdiff(first_frame, current_frame)
        threshold_frame=cv2.threshold(delta_frame, 20 ,255, cv2.THRESH_BINARY)[1]
    
        #Finds the contours in threshold
        (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
        #initially set current cars to 0
        current_cars = 0
    
        if previous_cars == None:
            previous_cars = current_cars

        #Loops in the contours array
        for c in contours:
            # contourArea() method filters out any small contours
            if cv2.contourArea(c) < 60:
                continue
                
            #So whenever there is a contour, increase current car by 1    
            current_cars = 1
        
        #Check if current cars is less than previous cars so a car has left the cropped area
        if current_cars < previous_cars:
            car_count += 1
        
        #Set the previous car as current car number
        previous_cars = current_cars
        
        
        cv2.putText(frame, "Cars going to city center: {}".format(car_count), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        #Show the webcam
        cv2.imshow('Webcam', frame)
    
    
        # Stop the program by pressing 'q'    
        if cv2.waitKey(1) == ord('q'):
            break
 
    #destroy all the windows
    video.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

    print("Total Cars: " + str(car_count))
    cars_per_min = round(car_count/minutes,2)
    print("Cars per minute: " + str(cars_per_min))

In [6]:
car_detection("Traffic_Laramie_1")

Total Cars: 6
Cars per minute: 2.02


In [7]:
car_detection("Traffic_Laramie_2")

Total Cars: 4
Cars per minute: 2.26
